# Финальное решение
Все вычисления проводились в collab и kaggle

In [1]:
!pip install -U bitsandbytes unsloth > /dev/null 2>&1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 17.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.3 MB/s eta 0:00:00:00:0100:01
   ━

## Загрузка и подготовка данных
Загрузка датасетов. Из обучающей выборки исключаются примеры с меткой 0.5, иначе модель начинает галлюцинировать на инференсе и качество хуже чем при обучении на всём датасете

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

# BASE_PATH = Path('/content/drive/MyDrive/ml/Tochka_bank')
BASE_PATH = Path('/kaggle/input/tochka-contest')
test_path = BASE_PATH / 'test.parquet'
train_path = BASE_PATH / 'train.parquet'

train_df = pd.read_parquet(train_path)
test_df = pd.read_parquet(test_path)

print(f"Train DataFrame loaded with {train_df.shape[0]} rows and {train_df.shape[1]} columns.")
print(f"Test DataFrame loaded with {test_df.shape[0]} rows and {test_df.shape[1]} columns.")
train_df.head()

stratify_col = (
    train_df["integrity"].astype(str) + "_" +
    train_df["factuality"].astype(str) + "_" +
    train_df["truthfulness"].astype(str)
)

mask = (train_df["integrity"] != 0.5) & (train_df["factuality"] != 0.5) & (train_df["truthfulness"] != 0.5)
clear_train_df = train_df[mask]
# Filter stratify_col to match the indices of clear_train_df
stratify_col_filtered = stratify_col.loc[clear_train_df.index]

train_part, val_part = train_test_split(
    clear_train_df,
    test_size=0.2,
    random_state=42,
    stratify=stratify_col_filtered
)

train_part = pd.concat([train_part, train_df[~mask]], ignore_index=True)
print(f"Train: {len(train_part)}")
print(f"Val:   {len(val_part)}")

Train DataFrame loaded with 15000 rows and 7 columns.
Test DataFrame loaded with 5000 rows and 1 columns.
Train: 13129
Val:   1871


## Динамическая генерация Few-Shot примеров
Здесь реализован алгоритм поиска самых коротких примеров текстов для каждого возможного паттерна оценок (например, 1-1-1, 0-0-0). Это позволяет экономить токены(в среднем тексты очень длинные), сохраняя при этом репрезентативность примеров в промпте.

In [5]:
def get_shortest_example(df, integrity, factuality, truthfulness):
    mask = (
        (df['integrity'] == integrity) &
        (df['factuality'] == factuality) &
        (df['truthfulness'] == truthfulness)
    )
    subset = df[mask].copy()

    if subset.empty:
        return None

    subset['text_len'] = subset['text'].str.len()
    shortest = subset.sort_values('text_len').iloc[0]
    return shortest

patterns = {
    '0_0_0': (0, 0, 0),
    '1_0_0': (1, 0, 0),
    '0_1_0': (0, 1, 0),
    '0_0_1': (0, 0, 1),
    '1_1_1': (1, 1, 1)
}

examples = {}

for name, (i, f, t) in patterns.items():
    row = get_shortest_example(train_part, i, f, t)
    if row is not None:
        combined_reasoning = (
            f"Integrity analysis: {row['integrity_reasoning']} "
            f"Factuality analysis: {row['factuality_reasoning']} "
            f"Truthfulness analysis: {row['truthfulness_reasoning']}"
        )

        examples[name] = {
            'text': row['text'],
            'reasoning': combined_reasoning,
            'integrity': int(row['integrity']),
            'factuality': int(row['factuality']),
            'truthfulness': int(row['truthfulness'])
        }
        print(f"Found {name}: len={len(row['text'])} chars")

Подбираем примеры минимальной длины...
Found 0_0_0: len=122 chars
Found 1_0_0: len=140 chars
Found 0_1_0: len=173 chars
Found 0_0_1: len=204 chars
Found 1_1_1: len=82 chars


## Инициализация модели и токенизатора
Загрузка 4-битной квантованной модели через Unsloth. Используется модель из моего репозитория, модель после обучения

In [6]:
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2096
dtype = None
load_in_4bit = True

model_name = "go2swim/tochkabank_lora_4b"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Модель уже загружается обученная
# model = FastLanguageModel.get_peft_model(
#     model,
#     r=16,
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
#                     "gate_proj", "up_proj", "down_proj"],
#     lora_alpha=16,
#     lora_dropout=0,
#     bias="none",
#     use_gradient_checkpointing="unsloth",
#     random_state=3407,
#     use_rslora=False,
#     loftq_config=None,
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-02-03 09:10:10.124107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770109810.313523      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770109810.369586      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770109810.835066      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770109810.835102      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770109810.835105      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
Загружаем модель для обучения: go2swim/tochkabank_lora_4b
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Подготовка промптов и функции форматирования
Формирование системного промпта с жесткими правилами (JSON output) и добавление few-shot примеров, найденных ранее. Функция formatting_prompts_func собирает диалог для обучения.

In [22]:
import json

def create_prompt_with_examples(target_text, examples_dict):
    system_message = """Ты эксперт по оценке качества текстов для обучения LLM.
        Твоя задача — проанализировать текст и выставить оценки (0 или 1) по трем критериям.
        Задача строго бинарная: НЕЛЬЗЯ использовать значение 0.5.

        КРИТЕРИИ:
        1. Integrity (Целостность): 1 - связный текст; 0 - бессвязный, спам.
        2. Factuality (Фактологичность): 1 - есть факты/описания миров; 0 - нет фактов, эмоции.
        3. Truthfulness (Правдивость): 1 - не противоречит реальному миру; 0 - фантастика/противоречия.

        ФОРМАТ ОТВЕТА:
        Строго выведи JSON:
        ```json
        {"integrity": 0 или 1, "factuality": 0 или 1, "truthfulness": 0 или 1}
        ```"""

    messages = [{"role": "system", "content": system_message}]
    order = ['1_1_1', '0_0_0', '1_0_0', '0_0_1']

    for key in order:
        if key in examples_dict:
            ex = examples_dict[key]
            messages.append({"role": "user", "content": f'Текст: "{ex["text"]}"'})
            assistant_content = (
                f"Reasoning: {ex['reasoning']}\n"
                "```json\n{\n"
                f'  "integrity": {ex["integrity"]},\n'
                f'  "factuality": {ex["factuality"]},\n'
                f'  "truthfulness": {ex["truthfulness"]}\n'
                "}\n```"
            )
            messages.append({"role": "assistant", "content": assistant_content})

    messages.append({"role": "user", "content": f'Текст: "{target_text}"'})

    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(batch):
    output_texts = []
    texts = batch['text']
    int_vals = batch['integrity']
    fact_vals = batch['factuality']
    truth_vals = batch['truthfulness']
    int_reas = batch['integrity_reasoning']
    fact_reas = batch['factuality_reasoning']
    truth_reas = batch['truthfulness_reasoning']

    for i in range(len(texts)):
        input_text = texts[i][:2000]
        prompt = create_prompt_with_examples(input_text, examples)

        combined_reasoning = (
            f"Integrity analysis: {int_reas[i]} "
            f"Factuality analysis: {fact_reas[i]} "
            f"Truthfulness analysis: {truth_reas[i]}"
        )

        json_response = {
            "integrity": int(int_vals[i]),
            "factuality": int(fact_vals[i]),
            "truthfulness": int(truth_vals[i])
        }

        assistant_response = (
            f"Reasoning: {combined_reasoning}\n"
            "```json\n"
            f"{json.dumps(json_response, indent=2)}\n"
            "```" + EOS_TOKEN
        )
        output_texts.append(prompt + assistant_response)

    return output_texts

## Обучение модели (SFT)
Настройка SFTTrainer. Используется оптимизатор adamw_8bit для экономии памяти и clean_train_mask для исключения примеров с меткой 0.5 из обучающей выборки.

In [6]:
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

clean_train_mask = (
    (train_part["integrity"] != 0.5) &
    (train_part["factuality"] != 0.5) &
    (train_part["truthfulness"] != 0.5)
)
final_train_df = train_part[clean_train_mask].copy()
train_dataset = Dataset.from_pandas(final_train_df)

output_dir = "/kaggle/working/lora_model_output"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    formatting_func=formatting_prompts_func,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=1,
        learning_rate=5e-6,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        save_steps=300,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=output_dir,
        report_to="none",
    ),
)

trainer.train()

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Фильтруем данные для обучения (убираем 0.5)...
Итого примеров для обучения: 7481
is_bfloat16_supported() False


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/7481 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
Начинаем обучение...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,481 | Num Epochs = 1 | Total steps = 936
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


...

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/go2swim/tochkabank_lora_4b
Загрузка токенизатора...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Готово! Проверьте репозиторий.


## Валидация с подсчетом метрик
Инференс на валидационном наборе данных. Включает:
Парсинг JSON-ответа от LLM с помощью регулярных выражений.
Подсчет метрики F1 "на лету", исключая из подсчета примеры с истинной меткой 0.5.

In [23]:
import torch
import re
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

def calculate_metrics_on_fly(targets_list, preds_list):
    cols = ['integrity', 'truthfulness', 'factuality']
    scores = []
    df_true = pd.DataFrame(targets_list)
    df_pred = pd.DataFrame(preds_list)
    f1_per_col = {}

    for col in cols:
        mask = df_true[col] != 0.5
        y_true = df_true.loc[mask, col].astype(int)
        y_pred = df_pred.loc[mask, col].astype(int)

        score = f1_score(y_true, y_pred, average='binary', zero_division=0) if len(y_true) > 0 else 0.0
        f1_per_col[col] = score
        scores.append(score)

    return np.mean(scores) if scores else 0, f1_per_col

def parse_llm_output(output_text):
    try:
        match = re.search(r'\{.*?\}', output_text, re.DOTALL)
        if match:
            data = json.loads(match.group(0))
            return {
                "integrity": int(data.get("integrity", 0)),
                "factuality": int(data.get("factuality", 0)),
                "truthfulness": int(data.get("truthfulness", 0))
            }
    except:
        pass
    return {"integrity": 0, "factuality": 0, "truthfulness": 0}

FastLanguageModel.for_inference(model)
all_targets, all_preds = [], []

for i, (uuid, row) in tqdm(enumerate(val_part.iterrows()), total=len(val_part)):
    text_truncated = row['text'][:2000]
    final_prompt = create_prompt_with_examples(text_truncated, examples)

    inputs = tokenizer([final_prompt], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=200, temperature=0.1,
            do_sample=False, pad_token_id=tokenizer.eos_token_id
        )

    response_text = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    parsed_scores = parse_llm_output(response_text)

    all_targets.append({'integrity': row['integrity'], 'factuality': row['factuality'], 'truthfulness': row['truthfulness']})
    all_preds.append(parsed_scores)

final_avg, final_details = calculate_metrics_on_fly(all_targets, all_preds)
print(f"Validation Average F1: {final_avg:.5f}")
print(f"Details: {final_details}")

Начинаем обработку 1871 строк...


Processing:   0%|          | 0/1871 [00:00<?, ?it/s]

Step 10: AVG F1: 0.9649 | Int: 1.000, Fact: 0.947, Tru: 0.947


KeyboardInterrupt: 

## Формирование Submission
Инференс на тестовом наборе данных.

In [ ]:
import os
import shutil

output_file = '/kaggle/working/submission.csv'
processed_uuids = set()

if os.path.exists(output_file):
    try:
        prev_df = pd.read_csv(output_file)
        processed_uuids = set(prev_df['uuid'].astype(str).values)
        print(f"Resuming. Processed: {len(processed_uuids)}")
    except:
        pass
else:
    with open(output_file, 'w') as f:
        f.write("uuid,integrity,truthfulness,factuality\n")

results_buffer = []
batch_size = 50

print(f"Total test samples: {len(test_df)}")

for i, (idx, row) in tqdm(enumerate(test_df.iterrows()), total=len(test_df)):
    current_uuid = str(row['uuid']) if 'uuid' in row else str(idx)
    if current_uuid in processed_uuids:
        continue

    text_truncated = row['text'][:2000]
    final_prompt = create_prompt_with_examples(text_truncated, examples)

    inputs = tokenizer([final_prompt], return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs, max_new_tokens=200, temperature=0.01,
            do_sample=False, pad_token_id=tokenizer.eos_token_id
        )

    response_text = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
    parsed = parse_llm_output(response_text)

    results_buffer.append({
        'uuid': current_uuid,
        'integrity': 1 if parsed['integrity'] == 1 else 0,
        'truthfulness': 1 if parsed['truthfulness'] == 1 else 0,
        'factuality': 1 if parsed['factuality'] == 1 else 0
    })

    if len(results_buffer) >= batch_size:
        pd.DataFrame(results_buffer).to_csv(output_file, mode='a', header=False, index=False)
        results_buffer = []

if results_buffer:
    pd.DataFrame(results_buffer).to_csv(output_file, mode='a', header=False, index=False)

print(f"Submission saved to {output_file}")

Test Inference:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Готово! Проверьте репозиторий.
submission saved: 49


Processing Files (0 / 0): |          |  0.00B /  0.00B            

...

New Data Upload: |          |  0.00B /  0.00B            

Готово! Проверьте репозиторий.
submission saved: 1849


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Готово! Проверьте репозиторий.
submission saved: 1899
